In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

In [2]:
from utils import get_cuda_info

get_cuda_info()

PyTorch version: 2.5.1+cu118
**********
_CUDA version: 
CUDA version:
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Wed_Oct_30_01:18:48_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.6, V12.6.85
Build cuda_12.6.r12.6/compiler.35059454_0

**********
CUDNN version: 90100
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 4070 Ti SUPER


## Zdobycie danych

In [3]:
from utils import load_data, convert_landmarks_to_eye_movements

all_data, all_labels = load_data('silesian_deception')

In [4]:
converted_data, blinks = convert_landmarks_to_eye_movements(all_data)

## Ograniczenie do pierwszych N klatek i normalizacja per próbkę

In [5]:
import numpy as np

def normalize_sample(data):
    min_val = np.min(data, axis=(0, 1), keepdims=True)
    max_val = np.max(data, axis=(0, 1), keepdims=True)
    normalized = (data - min_val) / (max_val - min_val + 1e-8)
    return normalized


In [26]:
normalized_cut_data = np.array([normalize_sample(d[:2000, :]) for d in converted_data], dtype=object)
blinks = np.array([d[:2000] for d in blinks], dtype=object)

In [32]:
combined_data = np.array([np.concatenate((a, b.reshape(-1, 1)), axis=-1) for a, b in zip(normalized_cut_data, blinks)], dtype=object)

## Preprocessing danych

In [33]:
from utils import preprocess_data

X_train, X_val, X_test, y_train, y_val, y_test = preprocess_data(combined_data, all_labels, binarize_labels=False)

In [34]:
print(X_train.shape)
print(y_train.shape)

torch.Size([700, 2000, 3])
torch.Size([700])


In [35]:
from utils import get_class_distribution

get_class_distribution(all_labels)

===> Class distribution <===
0: 301
1: 699
=============><=============


In [36]:
print("y_train")
get_class_distribution(np.array(y_train))
print("y_val")
get_class_distribution(np.array(y_val))
print("y_test")
get_class_distribution(np.array(y_test))

y_train
===> Class distribution <===
0.0: 222
1.0: 478
=============><=============
y_val
===> Class distribution <===
0.0: 40
1.0: 110
=============><=============
y_test
===> Class distribution <===
0.0: 39
1.0: 111
=============><=============


# MODEL TORCH

## Zbudowanie modelu ekstrakcji cech

In [37]:
import torch
import torch.nn as nn

In [43]:
class EyeDirectionAndBlinkTransformerClassifier(nn.Module):
    def __init__(self, d_model=64, nhead=4, num_layers=2, dim_feedforward=128, dropout=0.1):
        super().__init__()
        
        self.input_proj = nn.Linear(3, d_model)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.cls_head = nn.Sequential(
            nn.Linear(d_model, 32),
            nn.ReLU(),
            nn.Linear(32, 1)  # Binary classification
        )

    def forward(self, x):
        # x: [batch, seq_len, 2]
        x = self.input_proj(x)  # -> [batch, seq_len, d_model]
        x = self.transformer_encoder(x)  # -> [batch, seq_len, d_model]
        x = x.mean(dim=1)  # Global average pooling over time
        out = self.cls_head(x)  # -> [batch, 1]
        return out


In [ ]:
from torch.optim import Adam 

model = EyeDirectionAndBlinkTransformerClassifier()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pos_weight = torch.tensor([(len(y_train) - y_train.sum()) / y_train.sum()]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = Adam(model.parameters(), lr=1e-4)

## Trening modelu

In [40]:
from torch.utils.tensorboard import SummaryWriter

RUNS_FOLDER_PATH = os.path.abspath('runs')
writer_path = os.path.join('runs',  model.__class__.__qualname__, 'eye_movement')
writer = SummaryWriter(writer_path)

In [41]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [42]:
from utils.model_functions import train_torch_model_binary

train_torch_model_binary(model, criterion, optimizer, X_train, y_train, X_val, y_val, writer=writer, batch_size=128, unbalanced=True, show_prediction_stats=False, epochs=1000)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (256000x3 and 2x64)

## Ewaluacja modelu

In [107]:
from utils.model_functions import eval_torch_model_binary

eval_torch_model_binary(model, criterion, X_test, y_test)


                                          EPOCH STATISTICS                                          
Epoch       : 1
----------------------------------------------------------------------------------------------------
                                             VALIDATION                                             
----------------------------------------------------------------------------------------------------
Loss        : 2.0611
Accuracy    : 0.4667
Precision   : 0.5128
Recall      : 0.5149
F1 Score    : 0.4605
----------------------------------------------------------------------------------------------------
                                          VALIDATION EXTRA                                          
TP Rate     : 0.4019                                    FP Rate     : 0.3721



## Overfit

In [108]:
from utils.model_functions import overfit_model

overfit_model(model, criterion, optimizer, X_train, y_train, epochs=3000)


                                          EPOCH STATISTICS                                          
Epoch       : 1
----------------------------------------------------------------------------------------------------
                                             VALIDATION                                             
----------------------------------------------------------------------------------------------------
Loss        : 0.4454
Accuracy    : 0.4688
Precision   : 0.5290
Recall      : 0.5250
F1 Score    : 0.4640
----------------------------------------------------------------------------------------------------
                                          VALIDATION EXTRA                                          
TP Rate     : 0.3000                                    FP Rate     : 0.2500


                                          EPOCH STATISTICS                                          
Epoch       : 2
----------------------------------------------------------------------------

KeyboardInterrupt: 

# TODYNET

### Przygotowanie danych

In [73]:
X_train_np = X_train.numpy()
X_val_np = X_val.numpy()
X_test_np = X_test.numpy()
y_train_np = y_train.numpy()
y_val_np = y_val.numpy()
y_test_np = y_test.numpy()

In [74]:
TodyNet_DATA_PATH = os.path.join("..", "..", "src", "external", "TodyNet", "data", "UCR", "SILESIAN_DECEPTION_EYE_MOVEMENT")

os.makedirs(TodyNet_DATA_PATH, exist_ok=True)

In [75]:
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32).unsqueeze(1)  # adding channel dimension
X_val_tensor = torch.tensor(X_val_np, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32).unsqueeze(1)

# Save the data in PyTorch (.pt) format
torch.save(X_train_tensor, os.path.join(TodyNet_DATA_PATH, 'X_train.pt'))
torch.save(X_val_tensor, os.path.join(TodyNet_DATA_PATH, 'X_valid.pt'))
torch.save(X_test_tensor, os.path.join(TodyNet_DATA_PATH, 'X.pt'))

# Save the labels in PyTorch (.pt) format
torch.save(y_train, os.path.join(TodyNet_DATA_PATH, 'y_train.pt'))
torch.save(y_val, os.path.join(TodyNet_DATA_PATH, 'y_valid.pt'))
torch.save(y_test, os.path.join(TodyNet_DATA_PATH, 'y.pt'))

In [76]:
X_train_tensor.shape

torch.Size([700, 1, 2000, 2])

### Trening modelu [pool_ratio 0.8, ponieważ rozmiar danych jest zbyt duży na 0.2]

In [ ]:
# cd .\src\external\TodyNet\src\ & python train.py --dataset='SILESIAN_DECEPTION_EYE_MOVEMENT' --num_layers 1 --in_dim 16 --hidden_dim 16 --out_dim 16 --pool_ratio 0.0 --kern_size "3" --groups 1